# Pretrained Network | Feature Extractor

El objetivo de esta sección es ver como implementar transfer-learning cuando tenemos una cantidad de datos disponible limitada, pero el dominio del objetivo es similar a la red que queremos usar.

Congelaremos la parte de extracción de features de la red VGG16 entrenada en el dataset IMAGENET, y agregaremos nuestro propio clasificador, para después reentrenar la red con nuestro pequeño dataset.

Otro de los objetivos será mostrar como preprocesar nuestros datos para conseguir entrenar una red.

Notesé que ya no calculamos MEAN y STD para normalizar aquí, si no que usamos los valores estandard (calculados por la comunidad) para rapidez

Además no usaremos data-augmentation aqui.

In [ ]:
from torchvision import models

#Obtenemos los pesos de la red VGG16 entrenada en el dataset IMAGENET

weights = models.VGG16_Weights.IMAGENET1K_V1

#Como vemos aqui observamos los valores a los que nuestro dataset debe ser normalizado
# y preprocesado para que la red pueda usarlo.

preprocess = weights.transforms()
print(preprocess)
IMAGENET_MEAN = preprocess.mean
IMAGENET_STD  = preprocess.std

print(IMAGENET_MEAN)
print(IMAGENET_STD)


ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)
[0.485, 0.456, 0.406]
[0.229, 0.224, 0.225]


Como vemos arriba:

### crop_size=[224]

Es el tamaño final de la imagen que el modelo usa como entrada: 224×224 píxeles.
Se consigue aplicando transforms.CenterCrop(224) después del resize.

### resize_size=[256]

Antes del recorte, la imagen se redimensiona de forma que su lado más corto mida 256 píxeles, manteniendo su proporción.
Esto equivale a transforms.Resize(256)

También observamos la media y desviación estandard que usaremos para la normalización.

Este pipeline que hemos visto arriba, tenemos que replicarlo en nuestro dataset.

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Replicamos el pipeline de preprocesado de la red VGG16
base_tf = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BILINEAR), # Redimensiona la imagen
    transforms.CenterCrop(224), # Recorta la imagen
    transforms.ToTensor(), # Convierte la imagen a un tensor
    transforms.Normalize(weights.transforms().mean, weights.transforms().std), # Normaliza la imagen
])

root = "./data/01"
train_ds = datasets.ImageFolder(f"{root}/train", transform=base_tf)
valid_ds = datasets.ImageFolder(f"{root}/valid", transform=base_tf)
test_ds  = datasets.ImageFolder(f"{root}/test",  transform=base_tf)

train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=64, shuffle=False)
test_dl  = DataLoader(test_ds,  batch_size=64, shuffle=False)
